In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer

In [7]:
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[ 6.76568821e-02  6.34959638e-02  4.87131216e-02  7.93049783e-02
   3.74480784e-02  2.65277852e-03  3.93749699e-02 -7.09846150e-03
   5.93613759e-02  3.15369926e-02  6.00980893e-02 -5.29052094e-02
   4.06067669e-02 -2.59308480e-02  2.98428629e-02  1.12693268e-03
   7.35148638e-02 -5.03818430e-02 -1.22386590e-01  2.37028021e-02
   2.97264922e-02  4.24768180e-02  2.56337766e-02  1.99516071e-03
  -5.69190793e-02 -2.71597970e-02 -3.29035409e-02  6.60249069e-02
   1.19007207e-01 -4.58791442e-02 -7.26214573e-02 -3.25840265e-02
   5.23413531e-02  4.50553596e-02  8.25299881e-03  3.67024355e-02
  -1.39415739e-02  6.53918535e-02 -2.64271889e-02  2.06436394e-04
  -1.36643201e-02 -3.62810530e-02 -1.95043851e-02 -2.89738197e-02
   3.94270085e-02 -8.84090811e-02  2.62420205e-03  1.36713544e-02
   4.83062565e-02 -3.11565921e-02 -1.17329203e-01 -5.11691049e-02
  -8.85287896e-02 -2.18962971e-02  1.42986327e-02  4.44168411e-02
  -1.34815536e-02  7.43392482e-02  2.66382787e-02 -1.98762789e-02
   1.79191

In [8]:
len(embeddings)

2

In [9]:
df = pd.read_csv('../dataset/preprocessed_data.csv')
df.head()

,name,cleaned_summary
0,Taxi Blues,"shlykov, a hard-working taxi driver and lyosha..."
1,The Hunger Games,the nation of panem consists of a wealthy capi...
2,Narasimham,poovalli induchoodan is sentenced for six yea...
3,The Lemon Drop Kid,"the lemon drop kid , a new york city swindler,..."
4,A Cry in the Dark,seventh-day adventist church pastor michael ch...


In [10]:
summaries = df['cleaned_summary'].tolist()
summaries

["shlykov, a hard-working taxi driver and lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",
 'the nation of panem consists of a wealthy capitol and twelve poorer districts. as punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual hunger games. the tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. in her first reaping, 12-year-old primrose everdeen is chosen from district 12. her older sister katniss volunteers to take her place. peeta mellark, a baker\'s son who once gave katniss bread when she was starving, is the other district 12 tribute. katniss and peeta are taken to the capitol, accompanied by their frequently drunk mentor, past victor haymitch abernathy. he warns them about the "career" tributes who train intensively at special academies and almost always 

In [11]:
summary_embeddings = model.encode(summaries)

In [12]:
type(summary_embeddings)

numpy.ndarray

In [17]:
len(summary_embeddings[0])

384

In [18]:
import faiss
import numpy as np

In [19]:
embeddings = np.array(summary_embeddings).astype('float32')

In [20]:
index = faiss.IndexFlatL2(embeddings.shape[1])

In [21]:
index.add(embeddings)

In [22]:
faiss.write_index(index, "../embeddings/movie_summary_index.faiss")

In [82]:
load_index = faiss.read_index("../embeddings/movie_summary_index.faiss")

In [152]:
query = "steals information from his targets by entering their dreams."

In [153]:
query = query.lower().strip()
query

'steals information from his targets by entering their dreams.'

In [154]:
query_embedding = model.encode([query]).astype('float32')

In [155]:
D, I = load_index.search(query_embedding, k=5)

In [156]:
I

array([[38656, 41239, 19239, 24235, 27814]])

In [157]:
D

array([[0.8777969, 0.9991244, 1.0273656, 1.0436487, 1.0788851]],
      dtype=float32)

In [160]:
movie_index = int(I[0][1])

In [161]:
df.iloc[movie_index]

name                                                       Inception
cleaned_summary    former dream architect dominick "dom" cobb  an...
Name: 41239, dtype: object